In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

In [2]:
# Ignoring the warning related to Setting with copy.
pd.options.mode.chained_assignment = None

start_year = 2005

# Extract the current year
URL = 'https://www.resultados-futbol.com/ligue_1'
r = requests.get(URL)
page = r.content
soup = BeautifulSoup(page, 'html5lib')

current_year = int(soup.find('div', class_ = "titular-data").text.strip()[0:4])+1

# Define the countries and the years
Years = range(start_year,current_year+1)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

In [3]:
table_fifa_all_years = pd.read_excel('Table_fifa_all_years_edited.xlsx')
table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,236,"['FC Porto', 'F.C. Porto']",Benfica,16.0
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,234,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,237,"['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo...",Benfica,1.7
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,1889,"['Belenenses', 'CF Os Belenenses', 'Belém', 'C...",Boavista,0.4
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,1891,"['Nacional', 'Clube Desportivo Nacional', 'CD ...",Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2067,2022,France,Clermont Foot,72,71,72,71,/team/1815/clermont-foot/,1815,['Clermont Foot'],AS Saint-Étienne,4.5
2068,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,1,"['Arsenal', 'Arsenal FC']",Tottenham Hotspur,30.0
2069,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,17,['Southampton'],Portsmouth,7.5
2070,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,479,"['Osasuna', 'Club Atlético Osasuna', 'CA Osasu...",Athletic Club de Bilbao,4.8


In [4]:
table_results_all_years = pd.read_excel('Table_results_all_years_edited.xlsx')
table_results_all_years

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team,points_between_teams_home_team,points_between_teams_away_team
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,...,1.0,1.0,2,2,49,64,1,10,0,0
1,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,Finalized,1 - 0,...,3.0,0.0,3,5,61,61,7,13,0,0
2,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,...,3.0,0.0,3,4,37,61,9,10,0,0
3,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,Finalized,1 - 0,...,3.0,0.0,4,3,58,42,10,10,0,0
4,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,...,0.0,3.0,3,4,26,55,3,10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,06 Mar 22,2022-03-06,Ligue 1,National League,Saint-Étienne,Metz,Not played yet,-,...,NaN,NaN,3,3,18,20,6,1,17,5
37746,2022,France,21 Nov 21,2021-11-21,Ligue 1,National League,Metz,Girondins Bordeaux,Finalized,3 - 3,...,1.0,1.0,1,2,8,12,2,5,3,18
37747,2022,France,30 Oct 21,2021-10-30,Ligue 1,National League,Metz,Saint-Étienne,Finalized,1 - 1,...,1.0,1.0,2,2,6,5,3,2,11,11
37748,2022,France,20 Apr 22,2022-04-20,Ligue 1,National League,Girondins Bordeaux,Saint-Étienne,Not played yet,-,...,NaN,NaN,3,3,20,18,0,0,33,15


# Create dictionary to match team names

In [5]:
%%time

# create a dictionary whose keys are the team names in the results table
# and the values are the possible names in FIFA for each one of those teams

d = {}

# Build 2 lists: one with unique values of all teams in results table and
# another with unique values of all teams in fifa table

for year in Years:
    for country in Countries:
        results_teams_list_year_country = list(set(
            list(table_results_all_years[(table_results_all_years['Year'] == year) & 
                     (table_results_all_years['Country'] == country)]
                     ['Home_team']) +\
            list(table_results_all_years[(table_results_all_years['Year'] == year) & 
                     (table_results_all_years['Country'] == country)]
                     ['Away_team'])))

        fifa_teams_list_year_country = list(set(
            list(table_fifa_all_years[(table_fifa_all_years['Year'] == year) & 
                     (table_fifa_all_years['Country'] == country)]
                     ['Name'])))
        
        print(year)
        print(country)
        print('\n')

        
# Code to find a possible mismatch between the number of teams per year and
# per country in the results and fifa tables (if data is accurate, it should
# not happen)
        
        
        if len(results_teams_list_year_country) != len(fifa_teams_list_year_country):
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print(f'ERROR: {year},{country},\n\
            Number of teams results table: {len(results_teams_list_year_country)},\n\
            Number of teams fifa table: {len(fifa_teams_list_year_country)}')
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print('\n')
        
        else:

            # We create an auxiliar DataFrame (df_aux) in order to sort the list of results
            # teams per year and per country by wuzzyfuzzy score, starting on the highest
            # score. This will allow to start matching names that have a higher level of
            # confidence, and consequently will allow to have a shorter list of names to
            # match the last teams, whose level of confidence is lower.
            
                
            data = {'Results_name':results_teams_list_year_country}
            df_aux = pd.DataFrame(data)
            
            df_aux['Fifa_name_suggested'] = df_aux['Results_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[0])
    )
            
            df_aux['Fifa_name_suggested_fuzzywuzzy_score'] = df_aux['Results_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[1])
    )
            
            df_aux.sort_values(by = ['Fifa_name_suggested_fuzzywuzzy_score'], ascending = False,
                          inplace = True)
            
            results_teams_list_year_country = list(df_aux['Results_name'])
            
  
            # Here we look for the best match between a results team name and a fifa team
            # name. For that we run different loops with different levels of fuzzywuzzy
            # score, starting in the highest one.
            
            
            fuzzywuzzy_score_level = [100,90,80,70,60,50,40,30,20,10,0]
            
            for results_team in results_teams_list_year_country:
                
                if results_team in d.keys():
                    continue
                
                else:

                    for score_level in fuzzywuzzy_score_level:

                        fuzzywuzzy_min_score = score_level

                        fifa_team_suggested = process.extractOne(results_team, 
                                fifa_teams_list_year_country)[0]
                        fifa_team_suggested_score = process.extractOne(results_team, 
                                fifa_teams_list_year_country)[1]

                        
                        # We remove the fifa_team_suggested from the list so the same name is
                        # not suggested twice
                        if fifa_team_suggested_score >= fuzzywuzzy_min_score:
                            fifa_teams_list_year_country.remove(fifa_team_suggested)

                            if fifa_team_suggested not in d.values():
                                
                                # With .loc we will create a dataframe just showing the rows in
                                # which the the team name is the fifa_team_suggested and then
                                # with .iloc we will get all the names of the team that appear
                                # in the fifa table
                                if results_team not in d.keys():
                                    d[results_team] = table_fifa_all_years.loc[
                                        table_fifa_all_years['Name']==fifa_team_suggested]\
                                    ['Fifa_team_all_names'].iloc[0]

                                else:
                                    d[results_team].append(
                                table_fifa_all_years.loc[table_fifa_all_years['Name']==fifa_team_suggested]
                                     ['Fifa_team_all_names'].iloc[0])
                                

                            print('Results team:',results_team,'-->','Fifa team:',fifa_team_suggested,
                                  '-->',fifa_team_suggested_score)

                            break
            
            print('\n')
            print('-----------------------------')
            print('\n')

2005
Portugal


Results team: Sporting Braga --> Fifa team: Sporting Braga --> 100
Results team: Gil Vicente --> Fifa team: Gil Vicente --> 100
Results team: Rio Ave --> Fifa team: Rio Ave --> 100
Results team: Nacional --> Fifa team: Nacional --> 100
Results team: Boavista --> Fifa team: Boavista --> 100
Results team: Marítimo --> Fifa team: Marítimo --> 100
Results team: Moreirense --> Fifa team: Moreirense --> 100
Results team: Benfica --> Fifa team: Benfica --> 100
Results team: União de Leiria --> Fifa team: União Leiria --> 95
Results team: Os Belenenses --> Fifa team: Belenenses --> 95
Results team: Beira Mar SC --> Fifa team: SC Beira-Mar --> 95
Results team: Estoril --> Fifa team: Estoril Praia --> 90
Results team: Penafiel --> Fifa team: F.C. Penafiel --> 90
Results team: Porto --> Fifa team: FC Porto --> 90
Results team: Vitória Setúbal --> Fifa team: Vitória Futebol Clube --> 86
Results team: Vitória Guimarães --> Fifa team: Vitória SC --> 86
Results team: Sporting CP --> F

Results team: Desportivo Aves --> Fifa team: Deportivo Aves --> 97


-----------------------------


2007
Spain


Results team: Gimnàstic Tarragona --> Fifa team: Tarragona --> 90
Results team: Recreativo --> Fifa team: Recreativo de Huelva --> 90


-----------------------------


2007
England


Results team: Watford --> Fifa team: Watford --> 100
Results team: Sheffield United --> Fifa team: Sheffield United --> 100
Results team: Reading --> Fifa team: Reading FC --> 95


-----------------------------


2007
Italy


Results team: Catania --> Fifa team: Catania --> 100
Results team: Torino --> Fifa team: Torino --> 100


-----------------------------


2007
Germany


Results team: Alemannia Aachen --> Fifa team: Alemannia Aachen --> 100
Results team: Energie Cottbus --> Fifa team: FC Energie Cottbus --> 95


-----------------------------


2007
France


Results team: Lorient --> Fifa team: Lorient --> 100
Results team: Valenciennes --> Fifa team: Valenciennes FC --> 95
Results team: Se

Results team: Brighton & Hove Albion --> Fifa team: Brighton & Hove Albion --> 100
Results team: Huddersfield Town --> Fifa team: Huddersfield Town --> 100


-----------------------------


2018
Italy


Results team: SPAL --> Fifa team: Spal --> 100
Results team: Benevento --> Fifa team: Benevento --> 100


-----------------------------


2018
Germany




-----------------------------


2018
France


Results team: Amiens SC --> Fifa team: Amiens SC Football --> 90


-----------------------------


2019
Portugal


Results team: CD Santa Clara --> Fifa team: Santa Clara --> 95
Results team: Belenenses SAD --> Fifa team: Os Belenenses --> 85


-----------------------------


2019
Spain


Results team: Huesca --> Fifa team: SD Huesca --> 90


-----------------------------


2019
England




-----------------------------


2019
Italy




-----------------------------


2019
Germany




-----------------------------


2019
France


Results team: Nîmes --> Fifa team: Nîmes Olympique --> 90




In [6]:
d

{'Sporting Braga': "['Sporting Braga', 'SC Braga', 'Braga']",
 'Gil Vicente': "['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']",
 'Rio Ave': "['Rio Ave', 'Rio Ave FC']",
 'Nacional': "['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']",
 'Boavista': "['Boavista', 'Boavista Futebol Clube', 'Boavista FC']",
 'Marítimo': "['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal']",
 'Moreirense': "['Moreirense', 'Moreira de Cónegos', 'Moreirense FC']",
 'Benfica': "['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",
 'União de Leiria': "['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD']",
 'Os Belenenses': "['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses']",
 'Beira Mar SC': "['SC Beira-Mar', 'SC Beira Mar']",
 'Estoril': "['Estoril Praia', 'Estoril', 'GD Estoril-Praia']",
 'Penafiel': "['F.C. Penafiel', 'FC Penafiel']",
 'Porto': "['FC Porto', 'F.C. Porto']",
 'Vitória Setúbal': "['Vitória Futebol Clube', 'Vitór

In [7]:
# Manually correct the errors of the dictionary. These errors happen because one of the
# teams has a very different name in the results table compared to the fifa table, or
# because one team has an opponent team with a very similar name, so in these rare cases
# the library fuzzywuzzy does not work well.

print('Before manual correction:')
print('Milan:',d['Milan'])
print('Inter:',d['Inter'])
print('Fiorentina:',d['Fiorentina'])
print('Köln:',d['Köln'])
print('Feirense:',d['Feirense'])
print('Saint-Étienne:',d['Saint-Étienne'])
print('PSG:',d['PSG'])

d['Milan'] = table_fifa_all_years[table_fifa_all_years['Name']=='AC Milan']\
['Fifa_team_all_names'].iloc[0]
d['Inter'] = table_fifa_all_years[table_fifa_all_years['Name']=='Inter Milan']\
['Fifa_team_all_names'].iloc[0]
d['Fiorentina'] = table_fifa_all_years[table_fifa_all_years['Name']=='Firenze']\
['Fifa_team_all_names'].iloc[0]
d['Köln'] = table_fifa_all_years[table_fifa_all_years['Name']=='FC Cologne']\
['Fifa_team_all_names'].iloc[0]
d['Feirense'] = table_fifa_all_years[table_fifa_all_years['Name']=='F. Santa Maria da Feira']\
['Fifa_team_all_names'].iloc[0]
d['Saint-Étienne'] = table_fifa_all_years[table_fifa_all_years['Name']=='AS Saint-Etienne']\
['Fifa_team_all_names'].iloc[0]
d['PSG'] = table_fifa_all_years[table_fifa_all_years['Name']=='Paris Saint-Germain']\
['Fifa_team_all_names'].iloc[0]
d['Olympique'] = table_fifa_all_years[table_fifa_all_years['Name']=='Olympique de Marseille']\
['Fifa_team_all_names'].iloc[0]

print('\n')
print('After manual correction:')
print('Milan:',d['Milan'])
print('Inter:',d['Inter'])
print('Fiorentina:',d['Fiorentina'])
print('Köln:',d['Köln'])
print('Feirense:',d['Feirense'])
print('Saint-Étienne:',d['Saint-Étienne'])
print('PSG:',d['PSG'])

Before manual correction:
Milan: ['AC Milan', 'Milan']
Inter: ['Inter Milan', 'Inter']
Fiorentina: ['Firenze', 'Fiorentina', 'ACF Fiorentina']
Köln: ['Hertha BSC Berlin', 'Hertha BSC', 'Hertha Berlin']
Feirense: ['F. Santa Maria da Feira', 'CD Feirense']
Saint-Étienne: ['Paris Saint-Germain']
PSG: ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne']


After manual correction:
Milan: ['AC Milan', 'Milan']
Inter: ['Inter Milan', 'Inter']
Fiorentina: ['Firenze', 'Fiorentina', 'ACF Fiorentina']
Köln: ['FC Cologne', '1. FC Köln']
Feirense: ['F. Santa Maria da Feira', 'CD Feirense']
Saint-Étienne: ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne']
PSG: ['Paris Saint-Germain']


In [8]:
d

{'Sporting Braga': "['Sporting Braga', 'SC Braga', 'Braga']",
 'Gil Vicente': "['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']",
 'Rio Ave': "['Rio Ave', 'Rio Ave FC']",
 'Nacional': "['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']",
 'Boavista': "['Boavista', 'Boavista Futebol Clube', 'Boavista FC']",
 'Marítimo': "['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal']",
 'Moreirense': "['Moreirense', 'Moreira de Cónegos', 'Moreirense FC']",
 'Benfica': "['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",
 'União de Leiria': "['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD']",
 'Os Belenenses': "['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses']",
 'Beira Mar SC': "['SC Beira-Mar', 'SC Beira Mar']",
 'Estoril': "['Estoril Praia', 'Estoril', 'GD Estoril-Praia']",
 'Penafiel': "['F.C. Penafiel', 'FC Penafiel']",
 'Porto': "['FC Porto', 'F.C. Porto']",
 'Vitória Setúbal': "['Vitória Futebol Clube', 'Vitór

# Join Results table and Fifa table

In [9]:
# Add 2 auxiliar columns to table_results_all_years with all the fifa team names for both home
# and away team to help joining both tables

table_results_all_years['Home_team_fifa_team_all_names'] = table_results_all_years['Home_team'].apply(
        (lambda x: d[x])
    )

table_results_all_years['Away_team_fifa_team_all_names'] = table_results_all_years['Away_team'].apply(
        (lambda x: d[x])
    )

table_results_all_years

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team,points_between_teams_home_team,points_between_teams_away_team,Home_team_fifa_team_all_names,Away_team_fifa_team_all_names
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,...,2,2,49,64,1,10,0,0,"['Boavista', 'Boavista Futebol Clube', 'Boavis...","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
1,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,Finalized,1 - 0,...,3,5,61,61,7,13,0,0,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']","['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo..."
2,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,...,3,4,37,61,9,10,0,0,"['F.C. Penafiel', 'FC Penafiel']","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
3,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,Finalized,1 - 0,...,4,3,58,42,10,10,0,0,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']","['Belenenses', 'CF Os Belenenses', 'Belém', 'C..."
4,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,...,3,4,26,55,3,10,0,0,"['Estoril Praia', 'Estoril', 'GD Estoril-Praia']","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,06 Mar 22,2022-03-06,Ligue 1,National League,Saint-Étienne,Metz,Not played yet,-,...,3,3,18,20,6,1,17,5,"['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS...","['FC Metz', 'Football Club de Metz']"
37746,2022,France,21 Nov 21,2021-11-21,Ligue 1,National League,Metz,Girondins Bordeaux,Finalized,3 - 3,...,1,2,8,12,2,5,3,18,"['FC Metz', 'Football Club de Metz']","['Bordeaux', 'FC Girondins Bordeaux', 'FCGB', ..."
37747,2022,France,30 Oct 21,2021-10-30,Ligue 1,National League,Metz,Saint-Étienne,Finalized,1 - 1,...,2,2,6,5,3,2,11,11,"['FC Metz', 'Football Club de Metz']","['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS..."
37748,2022,France,20 Apr 22,2022-04-20,Ligue 1,National League,Girondins Bordeaux,Saint-Étienne,Not played yet,-,...,3,3,20,18,0,0,33,15,"['Bordeaux', 'FC Girondins Bordeaux', 'FCGB', ...","['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS..."


In [10]:
%%time

# Join table_results_all_years and fifa table

# first we start adding to the results table the columns we need
# from the fifa table just for the home teams

df_join_only_home = pd.DataFrame()

Columns = ['ATT','MID','DEF','OVR','Rival_team','Budget_Mill_€']


for year in Years:
    for country in Countries:
        df_1 = table_results_all_years[(table_results_all_years['Year'] == year) &
                  (table_results_all_years['Country'] == country)
                  ]

        df_2 = table_fifa_all_years[(table_fifa_all_years['Year'] == year) &
                  (table_fifa_all_years['Country'] == country)
                  ]

        df_aux = pd.merge(df_1, df_2[['Fifa_team_all_names','ATT','MID','DEF',
                                      'OVR','Rival_team','Budget_Mill_€']], 
            left_on= 'Home_team_fifa_team_all_names', 
            right_on = 'Fifa_team_all_names')

        df_join_only_home = pd.concat([df_join_only_home,df_aux])



df_join_only_home.drop(['Fifa_team_all_names'], axis=1, inplace=True)

# Here we rename the new columns and add "Home_team" as a prefix
for column in Columns:
    df_join_only_home.rename({
    column:f'Home_team_{column}'
    }, axis=1, inplace=True)


# Now that we have added the home team data, let's add it for the away team.
# Here one of the tables to merge is the one we had generated for the
# home_team
    
df_join_total = pd.DataFrame()

for year in Years:
    for country in Countries:
        df_1 = df_join_only_home[(df_join_only_home['Year'] == year) &
                  (df_join_only_home['Country'] == country)
                  ]

        df_2 = table_fifa_all_years[(table_fifa_all_years['Year'] == year) &
                  (table_fifa_all_years['Country'] == country)
                  ]

        df_aux = pd.merge(df_1, df_2[['Fifa_team_all_names','ATT','MID','DEF',
                                      'OVR','Rival_team','Budget_Mill_€']], 
            left_on= 'Away_team_fifa_team_all_names', 
            right_on = 'Fifa_team_all_names')

        df_join_total = pd.concat([df_join_total,df_aux])


df_join_total.drop(['Fifa_team_all_names'], axis=1, inplace=True)

# Here we rename the new columns and add "Away_team" as a prefix
for column in Columns:
    df_join_total.rename({
    column:f'Away_team_{column}'
    }, axis=1, inplace=True)

df_join_total

CPU times: user 2.86 s, sys: 199 ms, total: 3.05 s
Wall time: 3.15 s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,Home_team_DEF,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,...,53,58,Vitória SC,0.375,80,74,74,75,Sporting Lisbon,5.0
1,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,...,55,61,FC Porto,0.100,80,74,74,75,Sporting Lisbon,5.0
2,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,...,52,59,Vitória Futebol Clube,2.000,80,74,74,75,Sporting Lisbon,5.0
3,2005,Portugal,10 Apr 05,2005-04-10,Liga Portuguesa,National League,Rio Ave,Benfica,Finalized,1 - 0,...,48,52,Benfica,2.900,80,74,74,75,Sporting Lisbon,5.0
4,2005,Portugal,19 Mar 05,2005-03-19,Liga Portuguesa,National League,Vitória Setúbal,Benfica,Finalized,0 - 2,...,56,61,Belenenses,0.350,80,74,74,75,Sporting Lisbon,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2022,France,03 Oct 21,2021-10-03,Ligue 1,National League,Stade Rennais,PSG,Finalized,2 - 0,...,75,76,FC Nantes,14.000,88,84,83,85,Olympique de Marseille,160.0
376,2022,France,22 Sep 21,2021-09-22,Ligue 1,National League,Metz,PSG,Finalized,1 - 2,...,73,73,AS Nancy,7.000,88,84,83,85,Olympique de Marseille,160.0
377,2022,France,29 Aug 21,2021-08-29,Ligue 1,National League,Stade de Reims,PSG,Finalized,0 - 2,...,75,73,ESTAC Troyes,8.500,88,84,83,85,Olympique de Marseille,160.0
378,2022,France,20 Aug 21,2021-08-20,Ligue 1,National League,Stade Brestois,PSG,Finalized,2 - 4,...,73,73,En Avant Guingamp,6.500,88,84,83,85,Olympique de Marseille,160.0


In [11]:
# In order to simply the code from now on, and considering we
# already have all the info we need in just 1 table, let's call
# simply df to the new data frame

df = df_join_total

# New column: Rivals

In [12]:
# Let's add another column that tells us if the game between the 2 teams
# is between rivals. If they are, the column will show 1, otherwise it
# will show 0.

df['Rivals'] = df.apply(
lambda x: 1 if (x['Home_team'] == x['Away_team_Rival_team'] or x['Away_team'] == x['Home_team_Rival_team']) else 0, axis = 1
)

# Let's drop some irrelevant columns from the df

df.drop(['Datetime_date_list','Competition_original_name_URL',
    'Home_score', 'Away_score',
    'Points_Home_Team', 'Points_Away_Team',
    'Home_team_fifa_team_all_names',
    'Away_team_fifa_team_all_names'], axis = 1, inplace = True)

df

,Year,Country,Date,Competition,Home_team,Away_team,Status,Result,1x2,number_of_games_last_days_home_team,...,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€,Rivals
0,2005,Portugal,22 May 05,National League,Boavista,Benfica,Finalized,1 - 1,x,2,...,58,Vitória SC,0.375,80,74,74,75,Sporting Lisbon,5.0,0
1,2005,Portugal,07 May 05,National League,Penafiel,Benfica,Finalized,1 - 0,1,3,...,61,FC Porto,0.100,80,74,74,75,Sporting Lisbon,5.0,0
2,2005,Portugal,24 Apr 05,National League,Estoril,Benfica,Finalized,1 - 2,2,3,...,59,Vitória Futebol Clube,2.000,80,74,74,75,Sporting Lisbon,5.0,0
3,2005,Portugal,10 Apr 05,National League,Rio Ave,Benfica,Finalized,1 - 0,1,2,...,52,Benfica,2.900,80,74,74,75,Sporting Lisbon,5.0,1
4,2005,Portugal,19 Mar 05,National League,Vitória Setúbal,Benfica,Finalized,0 - 2,2,4,...,61,Belenenses,0.350,80,74,74,75,Sporting Lisbon,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2022,France,03 Oct 21,National League,Stade Rennais,PSG,Finalized,2 - 0,1,4,...,76,FC Nantes,14.000,88,84,83,85,Olympique de Marseille,160.0,0
376,2022,France,22 Sep 21,National League,Metz,PSG,Finalized,1 - 2,2,2,...,73,AS Nancy,7.000,88,84,83,85,Olympique de Marseille,160.0,0
377,2022,France,29 Aug 21,National League,Stade de Reims,PSG,Finalized,0 - 2,2,3,...,73,ESTAC Troyes,8.500,88,84,83,85,Olympique de Marseille,160.0,0
378,2022,France,20 Aug 21,National League,Stade Brestois,PSG,Finalized,2 - 4,2,2,...,73,En Avant Guingamp,6.500,88,84,83,85,Olympique de Marseille,160.0,0


In [13]:
df.to_excel(
    'Merged_table_from_2005.xlsx', index = False)